# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())
# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
    # Join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = []
for f in file_path_list:
# reading csv file    
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
 # extracting each data row one by one and append it        
        for line in csvreader:
            full_data_rows_list.append(line) 

# Create smaller event data csv file used to insert data into the Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)
with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [4]:
# Check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# Make connection to a Cassandra instance on your local machine (127.0.0.1)
from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
      { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
    

#### Set Keyspace

In [7]:
session.set_keyspace('udacity')


## Queries to ask the following three questions of the data

### Query-1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
For this query-1, I am creating a table with composite primary key which includes sessionId as partition key and itemInsession as clustering key 

    






In [8]:
# Create table (artist_song_for_session) for query 1
query = "CREATE TABLE IF NOT EXISTS session_item "
query = query + "(sessionId text, itemInSession text, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession))"
session.execute(query)

In [9]:
# Insert data into table for query 1
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip heade
    for line in csvreader:
        query = "INSERT INTO session_item (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### SELECT to verify that the data have been inserted into each table

In [10]:
query = "SELECT artist, song, length from session_item WHERE sessionId=338 AND itemInSession=4"
rows = session.execute(query)
for row in rows:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### Query-2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
For this query-2, I am creating a table with composite primary key which includes (userid and sessionId) as partition key and itemInsession as clustering key 

In [11]:
# Create table (artist_song_user_for_user_session) 
query = "CREATE TABLE IF NOT EXISTS user_session "
query = query + "(userId text, sessionId text, itemInSession text, artist text, song text, firstName text, lastName text, PRIMARY KEY ((userId, sessionId), itemInSession))"
session.execute(query)

# Insert values from the csv file into the newly created table
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)# skip header
    for line in csvreader:
        query = "INSERT INTO user_session (userId, sessionId, artist, song, itemInSession, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), line[0], line[9], int(line[3]), line[1], line[4]))

**SELECT to verify that the data have been inserted into each table


In [12]:

query = "SELECT artist, song, firstName, lastName FROM user_session WHERE userId = '10' AND sessionId = '182'"
rows = session.execute(query)
for row in rows:
    print(row.artist, row.song, row.firstname, row.lastname)                    

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


## Query-3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
For this query-3, I am creating a table with composite primary key which includes sessionId as partition key and itemInsession as clustering key 

In [14]:
# Create table (user_for_song) 
query = "CREATE TABLE IF NOT EXISTS song "
query = query + "(song text, userId int,firstName text, lastName text, PRIMARY KEY (song, userId))"
session.execute(query)

# Insert data from the csv file into the newly created table
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)# skip header
    for line in csvreader:
        query = "INSERT INTO song (song, userId, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

**SELECT to verify that the data have been inserted into each table**

In [15]:
# Perform query 3
query = "SELECT firstName, lastName FROM song WHERE song='All Hands Against His Own'"
rows = session.execute(query)
for row in rows:
    print(row.firstname, row.lastname)        

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [16]:
query = "drop table session_item"
session.execute(query)
query = "drop table user_session"
session.execute(query)
query = "drop table song"
session.execute(query)


### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()